In [1]:
import ROOT as ROOT
import os
import sys, optparse,glob
from array import array
import math
from collections import OrderedDict

ROOT.gSystem.Load("/Users/ptiwari/hep_software/ExROOT/ExRootAnalysis/libExRootAnalysis.so")

ROOT.gStyle.SetFrameLineWidth(3)
ROOT.gStyle.SetOptTitle(0)
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetLegendBorderSize(0)
ROOT.gStyle.SetFillColor(2)
ROOT.gStyle.SetLineWidth(1)
ROOT.gStyle.SetHistFillStyle(2)

Welcome to JupyROOT 6.14/02


In [2]:
dirpath = '/Users/ptiwari/cernbox/LHE_2HDMa_2017_bbDM/rootLHEFiles/varyMchi'
filelist = [f for f in os.listdir(dirpath) if os.path.isdir(os.path.join(dirpath, f))]
runs = []
mydict = {}
for f in filelist:
    if 'MH3_400_MH4_200' in f:
        print(f.split('_')[-1])
        mydict.update({f.split('_')[-1]:f})
fl =  OrderedDict(sorted(mydict.items()))

leg_entry=[]
for i in fl:
    runs.append(mydict[i])
    leg_entry.append('M_{#chi} = '+i)
print(runs)

print(leg_entry)
    
hists=[]


for i in runs:
    lheROOT = ROOT.TChain("LHEF")
    lheROOT.Add(dirpath+'/'+i+'/cmsgrid_final.root')
    h_genMET=ROOT.TH1F('genMET',"",100,0,1000)
    genMET=[]
    NEntries = lheROOT.GetEntries()
    for ievent in range(NEntries):
        if ievent%1000==0: print ("Processed "+str(ievent)+" of "+str(NEntries)+" events.")
        lheROOT.GetEntry(ievent)
        ## Get all relevant branches
        event      = lheROOT.__getattr__('Event')
        num_event   = lheROOT.__getattr__('Event_size')
        num_part   = lheROOT.__getattr__('Particle_size')
        particle   = lheROOT.__getattr__('Particle')
        weight = event[0].Weight
        chi = ROOT.TLorentzVector()
        chibar = ROOT.TLorentzVector()
        for ipart in range(num_part):
            if weight > 0:
                if particle[ipart].PID == 52:
                    chi    = ROOT.TLorentzVector(particle[ipart].Px,particle[ipart].Py,particle[ipart].Pz,particle[ipart].E)
                if particle[ipart].PID == -52:
                    chibar = ROOT.TLorentzVector(particle[ipart].Px,particle[ipart].Py,particle[ipart].Pz,particle[ipart].E)
        genMET.append((chi+chibar).Pt())
        

    for met in genMET:
        h_genMET.Fill(met)
    hists.append(h_genMET)

c = ROOT.TCanvas()
legend=ROOT.TLegend(.53,.59,.87,.89)
legend.SetTextSize(0.038)

latex = ROOT.TLatex();
latex.SetNDC();
latex.SetTextSize(0.04);
latex.SetTextAlign(31);
latex.SetTextAlign(11);

latex1 = ROOT.TLatex();
latex1.SetNDC();
latex1.SetTextSize(0.04);
latex1.SetTextAlign(31);
latex1.SetTextAlign(11);

for hist in range(len(hists)):
    if hist==0:
        hists[hist].SetXTitle("genMET[GeV]")
        hists[hist].SetYTitle("Events")
        hists[hist].SetLineColor(hist+1)
        hists[hist].Rebin(5)
        hists[hist].SetLineWidth(3)
        hists[hist].Scale(1/hists[hist].Integral())
        hists[hist].SetMaximum(2)
        legend.AddEntry(hists[hist],leg_entry[hist],"L")
        hists[hist].Draw('HIST')
        hists[hist].GetXaxis().SetRangeUser(100,1000)

    else:
        hists[hist].SetXTitle("genMET[GeV]")
        hists[hist].SetYTitle("Events")
        colourcode = hist+1
        #if colourcode==5: colourcode = 6
        hists[hist].SetLineColor(colourcode)
        hists[hist].Rebin(5)
        hists[hist].SetLineWidth(3)
        hists[hist].Scale(1/hists[hist].Integral())
        hists[hist].SetMaximum(2)
        legend.AddEntry(hists[hist],leg_entry[hist],"L")
        hists[hist].Draw('HIST same')
        hists[hist].GetXaxis().SetRangeUser(100,1000)
        
latex.DrawLatex(0.11, .92, "2HDM+a   b#bar{b} + DM"+" "*40+"tan#beta = 35   M_{a}= 200 GeV")
latex1.DrawLatex(0.30, .80, "M_{A}= 400 GeV")
legend.Draw()
c.SetLogy()
#c.update()
c.SaveAs("Combined_mA_Mchi_400_200.pdf")

10
1
100
['bbDM_2HDMa_MH3_400_MH4_200_Mchi_1', 'bbDM_2HDMa_MH3_400_MH4_200_Mchi_10', 'bbDM_2HDMa_MH3_400_MH4_200_Mchi_100']
['M_{#chi} = 1', 'M_{#chi} = 10', 'M_{#chi} = 100']
Processed 0 of 5000 events.
Processed 1000 of 5000 events.
Processed 2000 of 5000 events.
Processed 3000 of 5000 events.
Processed 4000 of 5000 events.
Processed 0 of 5000 events.
Processed 1000 of 5000 events.
Processed 2000 of 5000 events.
Processed 3000 of 5000 events.
Processed 4000 of 5000 events.
Processed 0 of 5000 events.
Processed 1000 of 5000 events.
Processed 2000 of 5000 events.
Processed 3000 of 5000 events.
Processed 4000 of 5000 events.


Warning in <TROOT::Append>: Replacing existing TH1: genMET (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: genMET (Potential memory leak).
Info in <TCanvas::Print>: pdf file Combined_mA_Mchi_400_200.pdf has been created
